<font size = "8">__Space X Falcon 9 First Stage Landing Prediction__</Font>

<font size = "5">__Data Wrangling__</Font>

We will perform EDA to find some patterns in the data and determine what would be the label for training supervised models.

In the data set, there are several different cases where the booster did not land successfully. Sometimes a landing was attempted but failed due to an accident, such as, True Ocean means the mission outcome was successfully landed, and False Ocean means the mission outcome was unsuccessfully landed to a region of the ocean. True RTLS means the mission outcome was successfully landed to a ground pad, and False RTLS means the mission outcome was unsuccessfully landed. True ASDS means the mission outcome was successfully landed on a drone ship, while False ASDS means the mission outcome was unsuccessfully landed.

For this lab, we will mostly convert the outcomes into trainings with 1 meaning the booster successfully landed and 0 meaning unsuccessful.

<Font Size = "3">__Objectives__</Font>

Perform Exploratory Data Analysis and determine Training Labels.

<Font Size = "3">__Preparing Data Analysis__</Font>

In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_1.csv")
df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.000000,ISS,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.000000,LEO,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1011,-80.577366,28.561857


Identify and calculate the percentage of the missing values in each attribute.

In [5]:
df.isnull().sum()/len(df)*100

FlightNumber       0.000000
Date               0.000000
BoosterVersion     0.000000
PayloadMass        0.000000
Orbit              0.000000
LaunchSite         0.000000
Outcome            0.000000
Flights            0.000000
GridFins           0.000000
Reused             0.000000
Legs               0.000000
LandingPad        28.888889
Block              0.000000
ReusedCount        0.000000
Serial             0.000000
Longitude          0.000000
Latitude           0.000000
dtype: float64

Identify which columns are numerical and categorical.

In [7]:
df.dtypes

FlightNumber        int64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights             int64
GridFins             bool
Reused               bool
Legs                 bool
LandingPad         object
Block             float64
ReusedCount         int64
Serial             object
Longitude         float64
Latitude          float64
dtype: object

<Font Size = "4">__Step 1: Calculate the number of launches on each site__</Font>

The data contains several Space X launch facilities: Cape Canaveral Space Launch Complex 40 VAFB SLC 4E, Vadenberg Air Force Base Space Launch Complex 4E (SLC-4E), Kennedy Space Center Launch Complex 39A KSC LC 39A. The location of each launch is placed in the column LaunchSite.

Next, let's use the method value_counts() on the column to find the number of launches on each site. 

In [8]:
df['LaunchSite'].value_counts()

LaunchSite
CCAFS SLC 40    55
KSC LC 39A      22
VAFB SLC 4E     13
Name: count, dtype: int64

Each launch aims to a dedicated orbit. Here are some common types:

Low Earth Orbit (LEO): an Earth-centered orbit with an altitude of 2000 km or less or with at least 11.25 periods per day.

Very Low Earth Orbits: orbits with a mean altitude below 450 k. Operating these orbits can provide a number of benefits of Earth observation spacecraft operates closer to the observation.

GTO: a high Earth orbit that allows satellites to match Earth's rotation. Located at 22,236 miles above Earth's equator, this position is a valuable spot for monitoring weather, communications, and surveillance. Because the satellite orbits at the same speed that the Earth is turnging, the satellite seems to stay in place over a single longitude, although it may drift north to south.

SSO: a sun synchronous orbit also called a heliosynchronous orbit that is a nearly polar orbit around a planet, in which the satellite passes over a given point of the planet's surface at the same local mean solar time.

ES-L1: At the Lagrange points, the gravitational forces of the two large bodies cancel out in such a way that a small object placed in orbit there is in equilibrium relative to the center of mass of the large bodies. L1 is one point between the sun and the earth.

HEO: an elliptical orbit with high eccentricity, usually referring to one around Earth.

ISS: a modular space station in low Earth orbit that is a multinational collaborative project between five participating space agencies: NASA (United States), Roscosmos (Russia), JAXA (Japan), ESA (Europe), and CSA (Canada).

MEO: Geocentric orbits ranging in altitude from 2000 km to just below geosynchronous orbit at 35,786 kilometers. Also known as an intermediate circular orbit. THese are "most commonly at 20,200 kilometers or 20,650 kilometers with an orbital period of 12 hours.

HEO: Geocentric orbits above the altitude of geosynchronous orbit (35,786 km ir 22,236 mi)

GEO: a circular geosynchronous orbit 35,786 kiloeters above Earth's equator and following the direction of Earth's rotation.

PO: one type of satellites in which a satellite passes above or nearly above both poles of the body being orbited.

<Font Size = "4">__Step 2: Calculate the number and occurrence of each orbit.__</Font>

Use the method .value_counts() to determine the number and occurrence of each orbit in the Orbit column.

In [10]:
df['Orbit'].value_counts()

Orbit
GTO      27
ISS      21
VLEO     14
PO        9
LEO       7
SSO       5
MEO       3
ES-L1     1
HEO       1
SO        1
GEO       1
Name: count, dtype: int64

<Font Size = "4">__Step 3: Calculate the number and occurrence of mission outcome per orbit type.__</Font>

Use the method .value_counts() on the Outcome column to determine the number of landing_outcomes. Assign it to the variable landing_outcomes.

In [13]:
landing_outcomes = df['Outcome'].value_counts()
landing_outcomes

Outcome
True ASDS      41
None None      19
True RTLS      14
False ASDS      6
True Ocean      5
False Ocean     2
None ASDS       2
False RTLS      1
Name: count, dtype: int64

In [14]:
for i, outcome in enumerate(landing_outcomes.keys()):
    print(i, outcome)

0 True ASDS
1 None None
2 True RTLS
3 False ASDS
4 True Ocean
5 False Ocean
6 None ASDS
7 False RTLS


Next, we set the outcomes where the second stage did not land successfully.

In [15]:
bad_outcomes = set(landing_outcomes.keys()[[1,3,5,6,7]])
bad_outcomes

{'False ASDS', 'False Ocean', 'False RTLS', 'None ASDS', 'None None'}

<Font Size = "4">__Step 4: Create a landing outcome label from the Outcome column.__</Font>

Using the Outcome column, create a list where the element is zero if the corresponding row in Outcome is in the set bad_outcome; otherwise, it's one. Assign it to the variable landing_class.

In [16]:
landing_class = []
for x in df['Outcome']:
    if x in set(bad_outcomes):
        landing_class.append(0)
    else:
        landing_class.append(1)

This variable will represent the classification variable that represents the outcome of each launch. If the value is 0, the first stage did not land successfully. Otherwise, 1 means the first stage landed successfully.

In [17]:
df['Class'] = landing_class
df[['Class']].head(8)

,Class
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,1


In [18]:
df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


We can use this code to find the success rate.

In [19]:
df['Class'].mean()

0.6666666666666666

We can now export it to a CSV for the next section. In the next lab, we will provide data in a p